
# Diagnosing Heart Disease of Patient's based on Age, Resting Blood Pressure, Cholesterol, and Max Heart Rate.

## Group 30: Toby Lau, Neil Lin, Tanish Gupta, Kristy Fielding

### Background Information & Introduction

Heart disease is one of the leading causes of death around the world that is caused by a wide variety of factors. Common heart diseases include coronary heart disease, arrhythmias, strokes, etc. Throughout the 21st century, medical professionals have deduced how certain factors may give leeway to one having a certain type of heart disease and the extent of its danger to the human body (Centers for Disease Control and Prevention, 2023). In this project, we will try to determine whether a patient has heart disease given the relevant predictors. The dataset being used is a combination of datasets of patients who were suspected to have heart disease from Cleveland and Switzerland. The dataset contains 14 attributes that could contribute to a patient's heart disease. Of the 14, we hope to train a model that is able to classify these stages given some attributes.

**Research Question: How can age, resting blood pressure, cholesterol, and max heart rate diagnose heart disease in patients from the West?**


## Methods
To create the model, we will be combining two datasets from the same source so that we have more data to train and test on. In doing so, we capture a greater variety of data as patients from different parts of the world also have different lifestyles.

### Data reading and wrangling
1. Import the required libraries and load both datasets from the web.
3. Assign the correct column names and types to the dataset
4. Combine both datasets.
5. Wrangle the dataset into a tidy format.
6. Split the data into training and testing sets
7. Summarize and visualize data

### Hyperparameter Tuning
1. Set a seed.
2. Create a recipe that defines the "stage" column as our response variable and "age", "restbps", "chol", and "thalach" columns as predictors.
3. Standardize predictors using step_scale and step_center on all predictors.
4. Create a $k$-nn model  with the neighbors = tune() parameter.
5. Create a tibble with a single neighbors column containing numbers from 1 to 100.
6. Perform Cross-validation on 10 folds to tune the model and find the best $k$ value.
7. Plot the accuracy of the model against the number of neighbors to determine the $k$ value with the highest accuracy.
8. Using the $k$ value that obtained the highest accuracy, create a new $k$-nn model

### Evaluating
1. Test the model using our testing data set.
2. Check for over/underfitting.
3. Create a confusion matrix
4. Assess the model's performance using tidymodels.


### Data reading and wrangling
First, we import the necessary libraries.

In [ ]:
library(tidyverse)
library(ggplot2)
library(GGally)
library(tidymodels)
install.packages("themis")
install.packages("yardstick")
library(themis)
library(yardstick)

Then we read in both our datasets. They come from the same source but have different URLS. We know from the data source the name of each column thanks to a variable list, so we can read use "read_csv" with the col_names argument set to a list of column names.

In [ ]:
# importing the data from URL
cleveland_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data",
                          col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"))
hungarian_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data",
                          col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"))

# previewing the data
head(cleveland_data) 
head(hungarian_data) 

Figure 1\
Looking at figure 1, we can notice that some of the columns which are supposed to be exactly the same have different types. The data source also tells us the correct type of each column so we can mutate both datasets accordingly.

In [ ]:
# mutating all the data 
cleveland_data <- cleveland_data |> 
    mutate(age = as.integer(age), sex = as.factor(sex), cp = as.factor(cp), trestbps = as.integer(trestbps), 
    chol = as.integer(chol), fbs = as.factor(fbs), restecg = as.factor(restecg), thalach = as.integer(thalach),
    exang = as.factor(exang), oldpeak = as.integer(oldpeak), slope = as.factor(slope), ca = as.integer(ca), 
    thal = as.factor(thal), num = as.integer(num))

hungarian_data <- hungarian_data |> 
    mutate(age = as.integer(age), sex = as.factor(sex), cp = as.factor(cp), trestbps = as.integer(trestbps), 
    chol = as.integer(chol), fbs = as.factor(fbs), restecg = as.factor(restecg),  thalach = as.integer(thalach),
    exang = as.factor(exang), oldpeak = as.integer(oldpeak), slope = as.factor(slope), ca = as.integer(ca), 
    thal = as.factor(thal), num = as.integer(num))

# what the new data looks like
head(cleveland_data)
head(hungarian_data)

Figure 2\
Although unclean, the datasets in figure 2 should have the same type in each column which allows us to combine the two. We will call this new resulting dataset "data_all". We also don't need every single column the dataset provides, so we will also select only the ones we need.

In [ ]:
data_all <- bind_rows(cleveland_data, hungarian_data) |> 
    select(age, trestbps, chol, thalach, -ca, -thal, num)

head(data_all)
nrow(data_all)

Figure 3\
With all our data in one tibble, we can move on to the actual cleaning. First we see whether there are any null values.

In [ ]:
nulls <- data_all |> 
    map_df(\(c) sum(is.na(c)))

nulls

Figure 4\
According to the figure above, there are still a few null values in the "trestbps", "chol", and "thalach" columns. So we must filter those values out.

In [ ]:
data_clean <- data_all |> 
    filter(!is.na(trestbps)) |> 
    filter(!is.na(chol)) |> 
    filter(!is.na(thalach))

nrow(data_clean)

This leaves us with 573 rows in the dataset left but this should still be sufficient.

Now, we decided to count the number of observations for each "num" category. To check how balanced the dataset is.

In [ ]:
options(repr.plot.width = 6, repr.plot.height=5)

summary <- data_all |> 
    group_by(num) |> 
    summarize(count = n())

summary_plot <- summary |> 
    ggplot(aes(num, count)) +
    geom_bar(stat = "identity") +
    labs(x = "Value in `num` column", y = "Number of observations") +
    ggtitle("Figure 5: Number of Observations per Factor in Num Column")

summary_plot

Figure 5\
Unfortunately, according to figure 5, our dataset is extremely imbalanced. So we've decided to create a new class that merges values of num from 1 to 4. 

According to the data source, values of 0 in the num column represent patients with <50% vessel diameter narrowing while values > 0 in the num column represent patients with >50% vessel diameter narrowing. Since mild heart disease is classified as <50% narrowing, moderate heart disease is classified as 50-80% narrowing, and severe heart disease is 80-100% narrowing (Cleveland Clinic). We can bucket the "num" values into respective "mild", "moderate", and "severe" classes into a new "stage" column. However, due to the class imbalance, we have decided to combine the "moderate" and "severe" classes. This corresponds to num values of 1-4. The following table represents how the new "stage" column relates to "num".

| Num value | Percentage blockage | Stage |
| -------- | ------- | ----|
| 0 | < 50% | Mild |
| 1-4 | > 50% | Moderate/Severe |

With only two classes, we will perform a binary classification instead.

In [ ]:
options(repr.plot.width = 7, repr.plot.height=5)

data_clean <- data_clean |> 
    mutate(stage = cut(num, breaks=c(-Inf,0,4), labels=c("Mild", "Moderate/Severe")))
head(data_clean)

data_clean_summary <- data_clean |> 
    group_by(stage) |> 
    summarize(count = n()) |> 
    ggplot(aes(stage, count)) +
    geom_bar(stat = "identity") +
    labs(x = "Stage of Heart Disease", y = "Number of Observations") +
    ggtitle("Figure 6: Number of observations per Factor in Stage Column")

data_clean_summary

Figure 7\
By combining the classes, we improve the balance of the classes slightly but not as much as we would like. We will most likely have to upsample when creating the model's recipe.

In [ ]:
set.seed(69)

split <- initial_split(data_clean, prop=0.75, strata = stage)
data_training <- training(split)
data_testing <- testing(split)

n_training <- nrow(data_training)
n_testing <- nrow(data_testing)
nrow(data_training)
nrow(data_testing)

Splitting our data into training and testing datasets using initial_split will ensure that bias and order were removed. There are 429 rows (75%) in the training set and 144 rows (25%) in the testing set. With this respective proportion, we will have enough data to sufficiently train our model and then utilize the testing set to evaluate its performance on unseen data. 

In [ ]:
training_summary <- data_training |> 
    group_by(stage) |> 
    summarize(count = n(), Prop = (n()/n_training)*100)

training_summary

Figure 8\
Within the training dataset, 58% of the data belongs to the "Mild" class while 46% of the data belongs to the "Moderate/Severe" class. In addition to the upsampling later, this should sufficiently reduce the bias in our model. To make sure the two classes are distinguishable we tabulate some each class' properties below.

In [ ]:
training_means <- data_training |> 
    group_by(stage) |> 
    summarize(mean_age = mean(age), mean_trestbps = mean(trestbps), mean_chol = mean(chol), mean_thalach = mean(thalach))

training_means

Figure 9\
According to the table above, there is quite a considerable difference in the mean of each predictor for each class. It shows that patients in the "Moderate/Severe" stage are older, have a higher resting heart rate, and higher levels of cholesterol but a lower max heart rate while the opposite is true for patients in the "Mild" stage.

### Summarizing and Visualizing the Data
With our training set, we can summarize and visualize our training data to uncover any underlying patterns in relation to our new "stage" column. The following code represents a ggpairs plot on our training data set that plots each predictor against each other. 

In [ ]:
options(options.repr.width = 10, options.repr.height = 10)

ggpair <- ggpairs(data_training, 
                  columns = 1:4, 
                  ggplot2::aes(colour = stage,alpha = 0.5), 
                  title="Figure 10: GGPairs plot of predictors")
ggpair

We can see that for some predictors there are fairly distinct regions that distinguish between the two classes we are predicting. For example, patient's with mild heart disease tend to have a higher maximum heart rate and patient's who are older tend to have more cholesterol.

The ggpairs plot also contains density plots to see how each the density of each predictor changes with class. This tells us on average, that patient's with mild heart disease are younger and patient's with a lower maximum heart rate have moderate/severe heart disease. Overall, our data should be sufficient in creating a viable model.

### Identifying the Ideal K Value

To identify the ideal K value, we create a model for each $k$ from 1 to 100 and select the $K$ value that obtained the highest accuracy. First, we need to create a recipe to upsample the data. To upsample data, we decided to use the SMOTE (Synthetic Minority Over-sampling Technique) since we believed that it should produce more reliable and varied data. Nevertheless, upsampled data may be inaccurate so we upsampled only up to 95% of the observations of the majority class. This means the minority class will have 95% of the number of rows of the majority class. We also decided to use a seed of 69 to ensure reproducibility.

In [ ]:
set.seed(69)

recipe <- recipe(stage ~ age + chol + trestbps + thalach, data = data_training) |>
    step_smote(stage, over_ratio=0.95) |>
    step_scale(all_predictors()) |> 
    step_center(all_predictors()) 

data_testing_new <- recipe |>
    prep() |>
    bake(NULL) |>
    group_by(stage) |>
    summarize(count = n()) |>
    ggplot(aes(stage, count)) +
    geom_bar(stat = "identity") +
    labs(x = "Class", y = "Number of observations") +
    ggtitle("Figure 11: Number of Observations per Class in Upsampled Testing Set")

data_testing_new

Now we can create our model. We decided to use a 10-fold cross validation as we believed this should be the best balance between accuracy and our computing power. Using a tibble with a single "neighbors" column containing values from 1 to 100, we created a model for each value and plotted its accuracy.

In [ ]:
set.seed(69)

k_vals <- tibble(neighbors = seq(1,100,1))

# using the features in the project proposal
tune_recipe <- recipe(stage ~ age + chol + trestbps + thalach, data = data_training) |>
    step_smote(stage) |>
    step_scale(all_predictors()) |> 
    step_center(all_predictors())

tune_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

tune_cv <- vfold_cv(data_training, v = 10, strata = stage)

tune_wf <- workflow() |>
    add_recipe(recipe) |>
    add_model(tune_spec)

tune_results <- tune_wf |>
    tune_grid(resamples = tune_cv, grid = k_vals) |>
    collect_metrics()

In [ ]:
options(repr.plot.width=7, repr.plot.height=5)


k_vs_accuracy <- tune_results |> 
    filter(.metric == "accuracy") |> 
    ggplot(aes(neighbors, mean)) +
    geom_point() +
    geom_line() +
    labs(x = "Number of neighbors", y = "Accuracy") +
    ggtitle("Figure 12: The Effect of the Number of Neighbors on Model Accuracy")

k_vs_accuracy

Looking at the plot above, we can see that the accuracy of the model is the best when $k$ is roughly 40. We can narrow down the value we need by outputting the table and arranging the "mean" column in descending order.

In [ ]:
ideal_k_table <- tune_results |> filter(.metric == "accuracy") |> arrange(desc(mean)) |> slice(1:5)
best_k <- ideal_k_table |> slice(1) |> select(neighbors) |> pull()
ideal_k_table

Figure 13\
We can see from the table above that 41 neighbors obtained the best accuracy of approximately 68%, meaning our ideal $k$ is 41. This $k$ value is saved to a variable called "best_k" for use later on.

### Building New Model
With our ideal $k$ value, we will build a new model and test it using our testing data set. 

In [ ]:
final_spec <- nearest_neighbor(weight_func= "rectangular", neighbors = best_k) |>
    set_engine("kknn") |>
    set_mode("classification")

final_workflow <- workflow() |>
    add_recipe(recipe)|>
    add_model(final_spec) |>
    fit(data = data_training)
final_workflow

### Testing New Model
Using our new model, we can be assured that it has the ideal $k$ value for diagnosing heart disease.

In [ ]:
set.seed(69)

predictions <- final_workflow |> predict(data_testing) |>
               bind_cols(data_testing)

# predictions 
metrics <- predictions |>
    metrics(truth = stage, estimate = .pred_class) |>
    filter(.metric == "accuracy")
metrics 

f_score <- f_meas(predictions, stage, .pred_class)
f_score

confusion_matrix <- predictions |>
    conf_mat(truth = stage, estimate = .pred_class)

confusion_matrix

Figure 14\
Based on the metrics above, our model was only somewhat accurate when used against our testing set. It obtained an accuracy of 67.4%, which is almost the desired 70% model accuracy usually aimed for. However, the model also achieved an f-score of roughly 72% ...

*Evaluate using accuracy, precision, recall, etc. Overdiagnose/Underdiagnose, how it can be used in the real-world etc.*

### Versatility

In [ ]:
set.seed(69)

long_beach_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.va.data",
                          col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"))

long_beach_data <- long_beach_data |> 
    mutate(age = as.integer(age), sex = as.factor(sex), cp = as.factor(cp), trestbps = as.integer(trestbps), 
    chol = as.integer(chol), fbs = as.factor(fbs), restecg = as.factor(restecg),  thalach = as.integer(thalach),
    exang = as.factor(exang), oldpeak = as.integer(oldpeak), slope = as.factor(slope), ca = as.integer(ca), 
    thal = as.factor(thal), num = as.integer(num))  |> select(age, trestbps, chol, thalach, -ca, -thal, num) 

long_beach_data <- long_beach_data |> 
    filter(chol != 0) |> 
    filter(thalach != "?") |> 
    filter(trestbps != "?") |> 
    filter(num != "?") |>
    mutate(stage = cut(num, breaks=c(-Inf,0,4), labels=c("Mild","Moderate/Severe")))

balance <- long_beach_data |> 
    group_by(stage) |> 
    summarize(count = n())
balance

predictions <- final_workflow |> 
    predict(long_beach_data) |>
    bind_cols(long_beach_data)

# predictions 
metrics <- predictions |>
    metrics(truth = stage, estimate = .pred_class) |>
    filter(.metric == "accuracy") 
metrics

# confusion matrix
confusion_matrix <- predictions |>
    conf_mat(truth = stage, estimate = .pred_class)
confusion_matrix

This model has a higher accuracy than before but imbalanced classes. This suggests that our model is good at detecting moderate/severe cases of heart disease, but less good at detecting milder cases or cases where there isn't any heart disease. Additionally, this dataset has less data than the previous datasets.

## Discussion

### Summary of findings
In summary, we found that we can predict the severity of heart disease with several predictors, with an accuracy of about 65%. We also tested our predictor on other datasets, which resulted in a slightly higher accuracy of 80%. In terms of precision and recall, the model performed okay on both. Since the model has a relatively low training and cross validation score, we can say that the best way to improve the model would probably be to increase the complexity of the model, because as of now it's not capturing all the variation in the dataset. 

### Expected findings
We expected to find that the model would be able to account for a greater amount of variation in the dataset. One possible reason why it's not doing the best could be because we didn't include all the features or because we simplified the target label (reducing it to binary classification when we could've done multi-class classification). We chose to simplify the classification such that a value in the dataset of 0 meant that the label was mild, and a value of 1-4 meant that the label was moderate to severe; however, we could have also done multiple classes which might have a better accuracy result. Additionally, if we had access to all the features and didn't simplify them during the preprocessing step of the data pipeline, the model might have generalized better to the cross validation data. 

### Future impact
If we can find a correlation between various factors and heart disease, this could impact future diagnosis, making the process faster and more accurate. A developed diagnosis pipeline could remove the need for a human doctor, which can be time consuming and expensive. Although the model isn't very accurate right now, a more complex model or a greater amount of data could lead to this model being available online for public use, allowing patients who are at risk for heart disease to find out sooner and with less hassle.

### Future questions
How can we increase the accuracy of the model, and how can this help patients with heart disease? How can changing the model architecture change it's performance on the dataset? How can the development of this model help more general cases of heart disease classification? How does the model perform when we provide more or less features?

## References

Carotid artery stenosis: Causes, symptoms and treatment. (n.d.). Cleveland Clinic. https://my.clevelandclinic.org/health/diseases/16845-carotid-artery-disease-carotid-artery-stenosis

Janosi, A., Steinbrunn, W., Pfisterer, M., & Detrano, R. (n.d.). Heart Disease - UCI Machine Learning Repository. UC Irvine Machine Learning Repository. https://archive.ics.uci.edu/dataset/45/heart+disease 

Centers for Disease Control and Prevention. (2023, May 15). Heart disease facts. Centers for Disease Control and Prevention. https://www.cdc.gov/heartdisease/facts.htm 